# Alternating Least Square Implementation

Matrix factorization by alternating least squares.

In [1]:
import numpy as np
import pandas as pd

## Intuition

First build some intuition by manually executing the 2 iterations

In [2]:
df = pd.read_csv("../../data/critics/critics.csv")
df.head()

,User,Movie,Rating
0,Lisa Rose,Lady in the Water,2.5
1,Lisa Rose,Snakes on a Plane,3.5
2,Lisa Rose,Just My Luck,3.0
3,Lisa Rose,Superman Returns,3.5
4,Lisa Rose,"You, Me and Dupree",2.5


In [3]:
user_product_matrix = df.pivot(index="User", columns="Movie", values="Rating").to_numpy()
user_product_matrix

array([[3. , nan, 3.5, 4. , 4.5, 2.5],
       [1.5, 3. , 3.5, 5. , 3. , 3.5],
       [nan, 3. , 4. , 5. , 3. , 3.5],
       [3. , 2.5, 3.5, 3.5, 3. , 2.5],
       [nan, 2.5, 3. , 3.5, 4. , nan],
       [2. , 3. , 4. , 3. , 3. , 2. ],
       [nan, nan, 4.5, 4. , nan, 1. ]])

We will try to factorize the matrix above with 3 latent factors.

### 1. Initialize User Matrix

In [4]:
# initialize user matrix

u_init = np.random.normal(0, 1/np.sqrt(3), size = (user_product_matrix.shape[0], 3))
u_init

array([[-0.10569108,  0.26511439,  1.01972518],
       [-0.43132784,  0.170877  , -0.01249452],
       [-1.16326879, -0.36218318,  0.41093088],
       [ 0.98078275,  0.41347612, -0.1854887 ],
       [ 1.18520702, -0.0931943 , -0.29204066],
       [-0.19458915, -0.13155516,  0.05557952],
       [-0.17723909,  0.00712118,  0.50824957]])

In [5]:
def calculate_ols_coefficients(X, y, l=0):
    X_2 = X.T @ X
    X_y = X.T @ y
    l_i = l * np.eye(X_2.shape[0])

    coeff = np.linalg.inv(X_2 + l_i) @ X_y
    return coeff

### 2. Calculate product matrix v with initialized user matrix

In [6]:
v = []

for j in range(user_product_matrix.shape[1]):
    dataset = np.hstack((u_init, np.expand_dims(user_product_matrix[:, j], axis = 1)))
    dataset = dataset[~np.isnan(dataset).any(axis = 1)]
    X = dataset[:, :-1]
    y = dataset[:, -1]
    coefficients = calculate_ols_coefficients(X, y, 0.1)
    v.append(coefficients)

v = np.array(v)
v


array([[ 0.65214355,  3.83931915,  1.76679219],
       [ 0.37012593, -0.36728096,  1.19014298],
       [ 1.12620662, -0.5016774 ,  4.46657946],
       [ 0.78780352,  0.11323872,  4.46442869],
       [ 1.73753525, -0.55284419,  4.33457816],
       [-2.56928179,  5.11698067,  1.04827525]])

### 3. (2nd iteration) calculate user matrix u with estimated v matrix in previous iteration

In [7]:
user_product_matrix_T = user_product_matrix.T
u = []

for j in range(0, user_product_matrix_T.shape[1]):
    dataset = np.hstack((v, np.expand_dims(user_product_matrix_T[:, j], axis = 1)))
    dataset = dataset[~np.isnan(dataset).any(axis = 1)]
    X = dataset[:, :-1]
    y = dataset[:, -1]
    coefficients = calculate_ols_coefficients(X, y, 0.1)
    u.append(coefficients)

u = np.array(u)
u

array([[ 0.19734865,  0.38243139,  0.87287739],
       [-0.93824551, -0.01362129,  1.16454913],
       [-1.80760603, -0.5191842 ,  1.40060496],
       [ 0.07553298,  0.36583266,  0.79579734],
       [ 0.32543897, -0.84980311,  0.66593658],
       [-0.15727174,  0.12233823,  0.85227528],
       [ 0.14820697,  0.07482668,  0.92176594]])

### 3. (2nd iteration) calculate product matrix v with estimated u matrix in 1st half of iteration

In [8]:
v = []

for j in range(0, user_product_matrix.shape[1]):
    dataset = np.hstack((u, np.expand_dims(user_product_matrix[:, j], axis = 1)))
    dataset = dataset[~np.isnan(dataset).any(axis = 1)]
    X = dataset[:, :-1]
    y = dataset[:, -1]
    coefficients = calculate_ols_coefficients(X, y, 0.1)
    v.append(coefficients)

v = np.array(v)
v


array([[ 1.24472715,  1.64304445,  2.38221125],
       [ 0.76170567, -0.0594712 ,  3.16112088],
       [ 1.02956796,  0.07185341,  4.15316102],
       [ 0.53323297, -0.27821804,  4.2775864 ],
       [ 1.76746075, -0.54301727,  4.1393926 ],
       [-1.26353228,  2.31083705,  1.77895833]])

Alternating least squares algorithm repeats this until the values of U and V matrices converge.

As you see below, u and v_T are then multiplied to reconstruct the user product rating matrix.

In [9]:
pred = u @ v.T
print(pred)
(pred).shape

[[ 2.95337533  2.88684887  3.85586319  3.73264194  3.75432135  2.18719274]
 [ 1.583962    2.96742373  3.86959381  4.48494578  3.16961054  3.2257113 ]
 [ 0.23351787  3.08149432  3.9185795   5.17177999  2.88470707  3.57583634]
 [ 2.59085465  2.55138898  3.40912715  3.3425873   3.22896574  2.16563161]
 [ 0.59522005  2.40353355  3.03974213  3.2585666   3.79323133 -1.19028573]
 [ 2.03554651  2.5670748   3.38650493  3.52778196  3.18349857  1.99758386]
 [ 2.50326199  3.02225361  3.98620806  4.00114414  4.03686892  1.62543117]]


(7, 6)

In [10]:
print(user_product_matrix)
print(user_product_matrix.shape)

[[3.  nan 3.5 4.  4.5 2.5]
 [1.5 3.  3.5 5.  3.  3.5]
 [nan 3.  4.  5.  3.  3.5]
 [3.  2.5 3.5 3.5 3.  2.5]
 [nan 2.5 3.  3.5 4.  nan]
 [2.  3.  4.  3.  3.  2. ]
 [nan nan 4.5 4.  nan 1. ]]
(7, 6)


With this intuition, the algorithm is replicated ideally until there is a convergence of a training error. Many loss functions can be used for the training. However, for starters and small use cases, using RMSE (calculated where corresponding entry exist in both predicted and actual matrices) is suffice.

In [11]:
# RMSE
float(np.sqrt(np.nanmean((user_product_matrix - pred) ** 2)))

0.3135816219517954

# Alternating least square algorithm

Now let's look at how different hyper paramters affect the model.

In [12]:
import sys
sys.path.append("../../src")

from recommendation.matrix_factorization.alternating_least_sqaures import ALS

In [13]:
ratings = pd.read_csv("../../data/movie_lens/rating.csv", nrows=900000)

In [14]:
%%time

als = ALS(
    n_features = 10,
    user_column_header = "userId",
    item_column_header = "movieId",
    rating_column_header = "rating",
    max_iter = 20
)

als.fit(rating_matrix = ratings)

INFO: Initializing user matrix
INFO: Start training
INFO: iteration 1: RMSE = 0.790997499680047
INFO: iteration 2: RMSE = 0.7422422887303891
INFO: iteration 3: RMSE = 0.719909898112242
INFO: iteration 4: RMSE = 0.7074798789692569
INFO: iteration 5: RMSE = 0.6989430885944551
INFO: iteration 6: RMSE = 0.6928308925583904
INFO: iteration 7: RMSE = 0.6884609707713909
INFO: iteration 8: RMSE = 0.6852563375897927
INFO: iteration 9: RMSE = 0.6828253345739962
INFO: iteration 10: RMSE = 0.6809365582799234
INFO: iteration 11: RMSE = 0.6794312787346508
INFO: iteration 12: RMSE = 0.6781980842452875
INFO: iteration 13: RMSE = 0.6771630885233478
INFO: iteration 14: RMSE = 0.6762762802994557
INFO: iteration 15: RMSE = 0.6755066985302216
INFO: iteration 16: RMSE = 0.6748347667264992
INFO: iteration 17: RMSE = 0.6742436314303372
INFO: iteration 18: RMSE = 0.6737193981552095
INFO: iteration 19: RMSE = 0.6732572901918471
INFO: iteration 20: RMSE = 0.6728450990046317


CPU times: user 1min 30s, sys: 2.77 s, total: 1min 32s
Wall time: 1min 24s


In [15]:
als.U.shape

(6034, 10)

In [16]:
als.V.shape

(13771, 10)

In [17]:
als.predict_rating(10, 145)

4.569839528655263

In [18]:
als.R[10, 145]

np.float64(5.0)

Lets try increasing the count of latent factor.

In [19]:
%%time

als = ALS(
    n_features = 100,
    user_column_header = "userId",
    item_column_header = "movieId",
    rating_column_header = "rating",
    max_iter = 20
)

als.fit(rating_matrix = ratings)

INFO: Initializing user matrix
INFO: Start training
INFO: iteration 1: RMSE = 0.465650849278539
INFO: iteration 2: RMSE = 0.36899803198335623
INFO: iteration 3: RMSE = 0.33012263743126735
INFO: iteration 4: RMSE = 0.3070242618272094
INFO: iteration 5: RMSE = 0.2911704127128393
INFO: iteration 6: RMSE = 0.2794368471798106
INFO: iteration 7: RMSE = 0.27031139319303216
INFO: iteration 8: RMSE = 0.2629523235512186
INFO: iteration 9: RMSE = 0.256850627568278
INFO: iteration 10: RMSE = 0.25168352079204676
INFO: iteration 11: RMSE = 0.24723545865900393
INFO: iteration 12: RMSE = 0.2433533277546159
INFO: iteration 13: RMSE = 0.23992384384369406
INFO: iteration 14: RMSE = 0.2368636312570362
INFO: iteration 15: RMSE = 0.23411191971979795
INFO: iteration 16: RMSE = 0.23162261369384707
INFO: iteration 17: RMSE = 0.229358706432097
INFO: iteration 18: RMSE = 0.2272897261472726
INFO: iteration 19: RMSE = 0.22539036623071773
INFO: iteration 20: RMSE = 0.223639427678969


CPU times: user 16min 41s, sys: 2.26 s, total: 16min 43s
Wall time: 4min 15s


In [20]:
als.U.shape

(6034, 100)

In [21]:
als.V.shape

(13771, 100)

In [22]:
als.predict_rating(10, 145)

5.04720781707735

In [23]:
als.R[10, 145]

np.float64(5.0)

We see here that more latent factors lead to less training errors; at the expense of training and inference time. In situations where large amounts of latent factors are called, stochastic gradient descent should be used to estimate the entries of the factor matrices.